In [1]:
import os
import io

import matplotlib.pyplot as plt
from PIL import Image
from google.cloud import vision

YOUR_PLATE = 'some_plate.jpg'
YOUR_SERVICE = 'YOUR_SERVICE'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = YOUR_SERVICE
client = vision.ImageAnnotatorClient()

with open(YOUR_PLATE, 'rb') as image_file:
    content = image_file.read()
image = vision.Image(content=content)

ImportError: ignored

In [ ]:
response = client.document_text_detection(image=image)

In [ ]:
im = Image.open(YOUR_PLATE)

plt.imshow(im)

for text in response.text_annotations:
    print(text.description)
    a = [(v.x, v.y) for v in text.bounding_poly.vertices]
    a.append(a[0])
    x, y = zip(*a)
    plt.plot(x, y, color='blue')

plt.show()

In [ ]:
response = client.object_localization(image=image)

In [ ]:
plt.imshow(im)

w, h = im.size
plates = []
for obj in response.localized_object_annotations:
    if obj.name == 'License plate':
        desc = f'{obj.name}: {obj.score:.2f}'
        a = [(v.x * w, v.y * h) for v in obj.bounding_poly.normalized_vertices]
        a.append(a[0])
        x, y = zip(*a)
        plates.append({'bound': [min(x), min(y), max(x), max(y)], 'coor': [x, y]})
        plt.plot(x, y, color='blue')
        plt.text(x[0], y[0], desc, color='red')

plt.show()

In [ ]:
for plate in plates:
    p = im.crop(plates[0]['bound'])
    buf = io.BytesIO()
    p.save(buf, format='JPEG')
    pi = vision.Image(content=buf.getvalue())
    response = client.text_detection(image=pi)
    r = {text.description.strip() for text in response.text_annotations}
    plate['text'] = r
    print(r)

In [ ]:
plt.imshow(im)

for plate in plates:
    desc = list(plate['text'])[0]
    plt.plot(plate['coor'][0], plate['coor'][1], color='blue')
    plt.text(x[0], y[0], desc, color='blue')

plt.axis('off')
plt.show()